Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your collaborators below:

In [ ]:
COLLABORATORS = ""

---

In [ ]:
import numpy as np

<div class="alert alert-info">**Hint**: Much of the material covered in this problem is introduced in the AIMA3 pp. 510-522 and 813-814 or AIMA2 pp. 492-504 and 722-724 readings. If you are having trouble with the questions in this notebook, this might be a good place to look.</div>

<div class="alert alert-danger">Note that in this problem, we will ask you to write down several equations. Your equations should be formatted using LaTeX. Refer back to Problem Set 0 if you forget how to format equations with LaTeX. **If your equations are not formatted with LaTeX, you will not receive full credit.**
</div>

  Bayesian networks provide an efficient way of representing a probability distribution. This made them very popular in artificial intelligence research, because they make it easy to handle uncertainty, as well as inferences both from cause to effect (making predictions) and effect to cause (forming explanations). These factors traditionally presented a challenge for AI approaches based on rules and symbols, such as production systems.
  
Read more about Bayesian networks here:
https://en.wikipedia.org/wiki/Bayesian_network

  The figure below shows a simple graphical model relating three variables. $X$ indicates whether the sprinklers were on last night, $Y$ indicates whether it rained last night, and $Z$ indicates whether the grass is wet. All three variables take on two values, with the value 1 being yes, and 0 being no.

![](images/graphmod.png)

*Hint: You can consult AIMA3 pp. 510-522 and 813-814, or AIMA2 pp. 492-504 and 722-724 for help.*

---
## Part A (1 point)

In this model, $X$, and $Y$ are <i>Bernoulli</i>-distributed random variables. A Bernoulli random variable takes a value of 1 with sucess probability $\theta$ and a value of 0 with failure probability $1-\theta$, where $\theta \in [0,1]$. For example, for $X$, we could write $P(X=1)=\theta_X$ and $P(X=0)=1-\theta_X$. Similarly, for $Y$, we can write $P(Y=1)=\theta_Y$ and $P(Y=0)=1-\theta_Y$.

(If you are familiar with the <i>Binomial</i> distribution, the <i>Bernoulli</i> is a special case, where $n$ = 1.)

Read more about these distributions following the links below:

<i>Bernoulli</i> https://en.wikipedia.org/wiki/Bernoulli_distribution

<i>Binomial</i> https://en.wikipedia.org/wiki/Binomial_distribution

<div class="alert alert-success">Complete the function `bernoulli` which returns the probability of a Bernoulli-distributed random variable (such as $X$ or $Y$) taking a particular value given success probability $\theta$.
</div>

In [ ]:
def bernoulli(value, theta):
    """
    Returns the probability that this variable will take on the 
    specified value given success probability theta.
    
    Parameters
    ----------
    
    value: integer
        Value of the random variable (either 0 or 1)
    theta : float
        Probability of success (between 0 and 1)
 
    Returns
    -------
    a float corresponding to the the probability of realizing the value

    """  
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
print("P(X=0 | theta=0.2) = " + str(bernoulli(0, .2)))
print("P(X=1 | theta=0.2) = " + str(bernoulli(1, .2)))
print("P(Y=0 | theta=0.4) = " + str(bernoulli(0, .4)))
print("P(Y=1 | theta=0.4) = " + str(bernoulli(1, .4)))

In [ ]:
# add your own test cases here!


In [ ]:
"""Check that `bernoulli` produces expected output."""
from nose.tools import assert_equal

for theta in np.linspace(0, 1, 100):
    assert_equal(bernoulli(0, theta), 1 - theta)
    assert_equal(bernoulli(1, theta), theta)

print("Success!")

---
## Part B (1.5 points)

In the current case, we will say that the probability that $X=1$ (that the sprinkler is on) is $0.6$, and the probability that $Y=1$ (that it rained) is $0.2$. In other words, $\theta_X = 0.6$ and $\theta_Y = 0.2$.

Below, we define two functions `p_x` and `p_y` by calling the `bernoulli` function you just wrote, using $0.6$ and $0.2$ as the theta values respectively. Additionally, we will say that the probability of $Z$ given $X$ and $Y$ (the probability of the state of the grass, given whether it rained and whether the sprinkler is on) is given in this table (Table 1):


<table class="table table-striped" style="width: 18em;">
    <thead>
    <tr>
        <th>$x$</th> 
        <th>$y$</th> 
        <th style="white-space: nowrap">$P(~Z=1~|~X=x,Y=y~)$</th> 
    </tr>
    </thead>
    <tbody>
    <tr>
        <td>0</td>
        <td>0</td>
        <td>0.05</td>
    </tr>
    <tr>
        <td> 0 </td>
        <td> 1 </td>
        <td> 1.0 </td>
    </tr>
    <tr>
        <td> 1 </td>
        <td> 0 </td>
        <td> 1.0 </td>
    </tr>
    <tr>
        <td> 1 </td>
        <td> 1 </td>
        <td> 1.0 </td>
    </tr>
    </tbody>
</table>

The provided function `p_z_given_xy` returns probability that 
$Z=z$ for a given combination of $x$ and $y$, following the above table:

In [ ]:
def p_x(x):
    """Computes P(X=x)"""
    return bernoulli(x, 0.6)

def p_y(y):
    """Computes P(Y=y)"""
    return bernoulli(y, 0.2)

def p_z_given_xy(z, x, y):
    """Computes P(Z=z | X=x, Y=y)"""
    if x == 0 and y == 0:
        return bernoulli(z, 0.05)
    else:
        return bernoulli(z, 1)

Given the above information about $P(X=x)$, $P(Y=y)$, and $P(Z=z\ |\ X=x,X=y)$, it is now possible to derive the joint probability distribution on $X$, $Y$, and $Z$ in order to populate the fourth column for the following table, Table 2. We'll walk through how we'll calculate this in the next few problems.
  
<table class="table table-striped" style="width: 15em;">
    <thead>
	<tr>
		<th> $x$ </th>
		<th> $y$ </th>
		<th> $z$ </th>
		<th> $P(~x,y,z~)$ </th>
	</tr>
    </thead>
    <tbody>
	<tr>
		<td> 0 </td>
		<td> 0 </td>
		<td> 0 </td>
		<td></td>
	</tr>
	<tr>
		<td> 0 </td>
		<td> 0 </td>
		<td> 1 </td>
		<td></td>
	</tr>
	<tr>
		<td> 0 </td>
		<td> 1 </td>
		<td> 0 </td>
		<td></td>
	</tr>
	<tr>
		<td> 0 </td>
		<td> 1 </td>
		<td> 1 </td>
		<td></td>
	</tr>
	<tr>
		<td> 1 </td>
		<td> 0 </td>
		<td> 0 </td>
		<td></td>
	</tr>
	<tr>
		<td> 1 </td>
		<td> 0 </td>
		<td> 1 </td>
		<td></td>
	</tr>
	<tr>
		<td> 1 </td>
		<td> 1 </td>
		<td> 0 </td>
		<td></td>
	</tr>
	<tr>
		<td> 1 </td>
		<td> 1 </td>
		<td> 1 </td>
	<td></td>
	</tr>
    </tbody>
</table> 

<div class="alert alert-success">
First, write down the equation for the joint probability distribution. Your answer should indicate how the probability can be calculated from $P(X=x)$, $P(Y=y)$, and $P(Z=z\ |\ X=x,Y=y)$. As a reminder, your answer MUST be formatted using LaTeX.
</div>

YOUR ANSWER HERE

<div class="alert alert-success">Now, complete the function `joint` so that it returns the joint probability as you defined it above. Use the functions `p_x`, `p_y`, and `p_z_given_xy` in the function definition, but do *not* call the function `bernoulli` (values of $\theta$ are defined in `p_x`, `p_y`, and `p_z_given_xy`). </div>


To emphasize, `p_x`, `p_y`, and `p_z_given_xy` are *functions* that you can call (choosing and using the correct parameters, of course).

In [ ]:
def joint(x, y, z):
    """
    Returns the joint probability distribution P(X=x, Y=y, Z=z)
    for the values x, y, and z.
    
    Parameters
    ----------
    x, y, z: integer
        Value of random variable X, Y, and Z, respectively
 
    Returns
    -------
    a float corresponding to the probability
    
    """
    # YOUR CODE HERE
    raise NotImplementedError()

Once you have completed your function, we can compute the values for Table 2 by applying the `joint` function to each combination of $x$, $y$, and $z$ values:

In [ ]:
for x in [0, 1]:
    for y in [0, 1]:
        for z in [0, 1]:
            print("P(X={}, Y={}, Z={}) = {}".format(x, y, z, joint(x, y, z)))

In [ ]:
# add your own test cases here!


In [ ]:
"""Check that `joint` produces expected output."""
from numpy.testing import assert_allclose

# first assume that p_x, p_y, and p_z_given_xy have the specified values
assert_allclose(joint(0, 0, 0), 0.304)
assert_allclose(joint(0, 0, 1), 0.016)
assert_allclose(joint(0, 1, 0), 0.0)
assert_allclose(joint(0, 1, 1), 0.08)
assert_allclose(joint(1, 0, 0), 0.0)
assert_allclose(joint(1, 0, 1), 0.48)
assert_allclose(joint(1, 1, 0), 0.0)
assert_allclose(joint(1, 1, 1), 0.12)

# check that the p_x, p_y, and p_z_given_xy functions are called
old_p_x = p_x
old_p_y = p_y
old_p_z_given_xy = p_z_given_xy
del p_x
del p_y
del p_z_given_xy
try:
    joint(0, 0, 1)
except NameError:
    pass
else:
    raise AssertionError("joint does not call one or more of the p_x, p_y, and/or p_z_given_xy functions")
finally:
    p_x = old_p_x
    p_y = old_p_y
    p_z_given_xy = old_p_z_given_xy
    del old_p_x
    del old_p_y
    del old_p_z_given_xy
    
# switch out p_x, p_y, and p_z_given_xy with alternative theta values
old_p_x = p_x
old_p_y = p_y
old_p_z_given_xy = p_z_given_xy
def p_x(x):
    return bernoulli(x, 0.5)
def p_y(y):
    return bernoulli(y, 0.3)
def p_z_given_xy(z, x, y):
    if x == 0 and y == 0:
        return bernoulli(z, 0.8)
    else:
        return bernoulli(z, 0.7)

try:
    assert_allclose(joint(0, 0, 0), 0.07)
    assert_allclose(joint(0, 0, 1), 0.28)
    assert_allclose(joint(0, 1, 0), 0.045)
    assert_allclose(joint(0, 1, 1), 0.105)
    assert_allclose(joint(1, 0, 0), 0.105)
    assert_allclose(joint(1, 0, 1), 0.245)
    assert_allclose(joint(1, 1, 0), 0.045)
    assert_allclose(joint(1, 1, 1), 0.105)
finally:
    p_x = old_p_x
    p_y = old_p_y
    p_z_given_xy = old_p_z_given_xy
    del old_p_x, old_p_y, old_p_z_given_xy
    
print("Success!")

---
## Part C (1 point)

Imagine that you observed the grass is wet in the morning ($Z=1$). What happens to your beliefs about the sprinklers being on and whether it rained? To answer this question we need to compute the conditional distribution over $X$ and $Y$ when $Z=z$, $P(X=x,Y=y\ |\ Z=z)$. We can calculate the conditional distribution from the joint distribution $P(X=x, Y=y, Z=z)$ as follows:

$$
P(X=x,Y=y\ |\ Z=z)=\frac{P(X=x, Y=y, Z=z)}{\sum_{x^\prime\in\{0, 1\}}\sum_{y^\prime\in \{0, 1\}} P(X=x^\prime, Y=y^\prime, Z=z)}
$$

Note that the denominator of this equation is equivalent to $P(Z=z)$.

<div class="alert alert-success">Complete the function `p_xy_given_z` so that it returns $P(X=x,Y=y~|~Z=z)$ for given values of $x$, $y$, $z$. Your answer should use the `joint` function you implemented above.
</div>

In [ ]:
def p_xy_given_z(x, y, z):
    """
    Returns the conditional probability distribution P(X=x, Y=y | Z=z)
    for the values x, y, and z.
    
    Parameters
    ----------
    x, y, z: integers
        values of random variables X, Y, and Z, respectively
 
    Returns
    -------
    a float corresponding to the probability
    
    """
    # YOUR CODE HERE
    raise NotImplementedError()

Once you have completed `p_xy_given_z`, we can compute the probabilities for each value of $x$, $y$, and $z$:

In [ ]:
for z in [0, 1]:
    for x in [0, 1]:
        for y in [0, 1]:
            print("P(X={}, Y={} | Z={}) = {}".format(x, y, z, p_xy_given_z(x, y, z)))

In [ ]:
# add your own test cases here!


In [ ]:
"""Check that `p_xy_given_z` produces expected output."""

assert_allclose(p_xy_given_z(0, 0, 0), 1.0, atol=1e-05)
assert_allclose(p_xy_given_z(0, 1, 0), 0.0, atol=1e-05)
assert_allclose(p_xy_given_z(1, 0, 0), 0.0, atol=1e-05)
assert_allclose(p_xy_given_z(1, 1, 0), 0.0, atol=1e-05)
assert_allclose(p_xy_given_z(0, 0, 1), 0.02298, atol=1e-05)
assert_allclose(p_xy_given_z(0, 1, 1), 0.11494, atol=1e-05)
assert_allclose(p_xy_given_z(1, 0, 1), 0.68965, atol=1e-05)
assert_allclose(p_xy_given_z(1, 1, 1), 0.17241, atol=1e-05)
    
# check that it calls joint
old_joint = joint
del joint
try:
    p_xy_given_z(0, 0, 1)
except NameError:
    pass
else:
    raise AssertionError("p_xy_given_z does not call the joint function")
finally:
    joint = old_joint
    del old_joint
    
# switch out p_x, p_y, and p_z_given_xy with alternative theta values
old_p_x = p_x
old_p_y = p_y
old_p_z_given_xy = p_z_given_xy
def p_x(x):
    return bernoulli(x, 0.5)
def p_y(y):
    return bernoulli(y, 0.3)
def p_z_given_xy(z, x, y):
    if x == 0 and y == 0:
        return bernoulli(z, 0.8)
    else:
        return bernoulli(z, 0.7)

try:
    assert_allclose(p_xy_given_z(0, 0, 0), 0.2641509433962263, atol=1e-05)
    assert_allclose(p_xy_given_z(0, 1, 0), 0.169811320754717, atol=1e-05)
    assert_allclose(p_xy_given_z(1, 0, 0), 0.39622641509433965, atol=1e-05)
    assert_allclose(p_xy_given_z(1, 1, 0), 0.169811320754717, atol=1e-05)
    assert_allclose(p_xy_given_z(0, 0, 1), 0.380952380952381, atol=1e-05)
    assert_allclose(p_xy_given_z(0, 1, 1), 0.14285714285714288, atol=1e-05)
    assert_allclose(p_xy_given_z(1, 0, 1), 0.33333333333333337, atol=1e-05)
    assert_allclose(p_xy_given_z(1, 1, 1), 0.14285714285714288, atol=1e-05)
finally:
    p_x = old_p_x
    p_y = old_p_y
    p_z_given_xy = old_p_z_given_xy
    del old_p_x, old_p_y, old_p_z_given_xy

print("Success!")

---
## Part D (1 point)

Now let's say that we are just interested in the belief about whether the sprinklers were on, given that we observed that the grass was wet ($Z=1$).

<div class="alert alert-success">Define $P(X=x\ |\ Z=z)$ following the same format and conventions as $P(X=x,Y=y~|~Z=z)$ above. Your answer should indicate how this can be calculated from $P(X=x, Y=y, Z=z)$. As a reminder, your answer MUST be formatted using LaTeX.</div>

YOUR ANSWER HERE

<div class="alert alert-success">Complete the function `p_x_given_z` so that it returns $P(X=x\ |\ Z=z)$ for given values of $x$ and $z$. Your function should call the `joint` function you implemented earlier.</div>

In [ ]:
def p_x_given_z(x, z):
    """
    Returns the marginal probability distribution P(X=x | Z=z) given 
    the values x and z.
    
    Parameters
    ----------
    x, z : integers
        values of the random variables X and Z, respectively
 
    Returns
    -------
    a float corresponding to the probability
    
    """
    # YOUR CODE HERE
    raise NotImplementedError()

Once you have implemented `p_x_given_z`, we can print out the probability table for all values of $X$ and $Z$:

In [ ]:
for z in [0, 1]:
    for x in [0, 1]:
        print("P(X={} | Z={}) = {}".format(x, z, p_x_given_z(x, z)))

In [ ]:
# add your own test cases here!


In [ ]:
"""Check that `p_x_given_z` produces expected output."""

assert_allclose(p_x_given_z(1, 1), 0.8620689, atol=1e-05)
assert_allclose(p_x_given_z(0, 1), 0.137931034483, atol=1e-05)
assert_allclose(p_x_given_z(1, 0), 0.0, atol=1e-05)
assert_allclose(p_x_given_z(0, 0), 1.0, atol=1e-05)
    
# check that it calls joint
old_joint = joint
del joint
try:
    p_x_given_z(0, 1)
except NameError:
    pass
else:
    raise AssertionError("p_x_given_z does not call the joint function")
finally:
    joint = old_joint
    del old_joint
    
# switch out p_x, p_y, and p_z_given_xy with alternative theta values
old_p_x = p_x
old_p_y = p_y
old_p_z_given_xy = p_z_given_xy
def p_x(x):
    return bernoulli(x, 0.5)
def p_y(y):
    return bernoulli(y, 0.3)
def p_z_given_xy(z, x, y):
    if x == 0 and y == 0:
        return bernoulli(z, 0.8)
    else:
        return bernoulli(z, 0.7)

try:
    assert_allclose(p_x_given_z(1, 1), 0.4761904761904762, atol=1e-05)
    assert_allclose(p_x_given_z(0, 1), 0.5238095238095238, atol=1e-05)
    assert_allclose(p_x_given_z(1, 0), 0.5660377358490567, atol=1e-05)
    assert_allclose(p_x_given_z(0, 0), 0.43396226415094336, atol=1e-05)
finally:
    p_x = old_p_x
    p_y = old_p_y
    p_z_given_xy = old_p_z_given_xy
    del old_p_x, old_p_y, old_p_z_given_xy

print("Success!")

<div class="alert alert-success">
Run the cell below to compute $P(~X=1~)$ and $P(~X=1~|~Z=1~)$. What does observing that the grass is wet ($Z=1$) do to the belief about whether the sprinklers were on ($X=1$)?
</div>

In [ ]:
print("P(X=1)       = {}".format(p_x(1)))
print("P(X=1 | Z=1) = {}".format(p_x_given_z(1, 1)))

YOUR ANSWER HERE

---
## Part E (1 point)

Imagine you got into your car, and heard on the radio that it rained last night ($Y=1$). How does this affect your beliefs about the sprinklers being on?

<div class="alert alert-success">Write down the equation for $P(X=x\ |\ Y=y,Z=z)$ following the same format and conventions as $P(X=x,Y=y~|~Z=z)$ above. Your answer should indicate how this can be calculated from $P(X=x, Y=y, Z=z)$. As a reminder, your answer MUST be formatted using LaTeX.</div>

YOUR ANSWER HERE

<div class="alert alert-success">Having written the equation above, complete the function `p_x_given_yz` so that it computes $P(~X=x~|~Y=y,Z=z~)$ for given values of $x$, $y$ and $z$. As before, your function should call the `joint` function you defined before.</div>

In [ ]:
def p_x_given_yz(x, y, z):
    """
    Returns the conditional probability distribution P(X=x | Y=y, Z=z)
    given the values x, y, and z.
    
    Parameters
    ----------
    x, y, z : integers
        values of the random variables X, Y, and Z
 
    Returns
    -------
    a float corresponding to the probability
    
    """
    # YOUR CODE HERE
    raise NotImplementedError()

After you have implemented `p_x_given_yz`, you can print out the full probability table:

In [ ]:
for z in [0, 1]:
    for y in [0, 1]:
        for x in [0, 1]:
            print("P(X={} | Y={}, Z={}) = {}".format(x, y, z, p_x_given_yz(x, y, z)))

In [ ]:
# add your own test cases here!


In [ ]:
"""Check that `p_x_given_yz` produces expected output."""

assert_allclose(p_x_given_yz(0, 0, 0), 1.0, atol=1e-05)
assert_allclose(p_x_given_yz(1, 0, 0), 0.0, atol=1e-05)
assert_allclose(p_x_given_yz(0, 1, 0), 0.0, atol=1e-05)
assert_allclose(p_x_given_yz(1, 1, 0), 0.0, atol=1e-05)
assert_allclose(p_x_given_yz(0, 0, 1), 0.03225806451612904, atol=1e-05)
assert_allclose(p_x_given_yz(1, 0, 1), 0.9677419354838709, atol=1e-05)
assert_allclose(p_x_given_yz(0, 1, 1), 0.4, atol=1e-05)
assert_allclose(p_x_given_yz(1, 1, 1), 0.6, atol=1e-05)
    
# check that it calls joint
old_joint = joint
del joint
try:
    p_x_given_yz(0, 1, 1)
except NameError:
    pass
else:
    raise AssertionError("p_x_given_yz does not call the joint function")
finally:
    joint = old_joint
    del old_joint
    
# switch out p_x, p_y, and p_z_given_xy with alternative theta values
old_p_x = p_x
old_p_y = p_y
old_p_z_given_xy = p_z_given_xy
def p_x(x):
    return bernoulli(x, 0.5)
def p_y(y):
    return bernoulli(y, 0.3)
def p_z_given_xy(z, x, y):
    if x == 0 and y == 0:
        return bernoulli(z, 0.8)
    else:
        return bernoulli(z, 0.7)

try:
    assert_allclose(p_x_given_yz(0, 0, 0), 0.4, atol=1e-05)
    assert_allclose(p_x_given_yz(1, 0, 0), 0.6, atol=1e-05)
    assert_allclose(p_x_given_yz(0, 1, 0), 0.5, atol=1e-05)
    assert_allclose(p_x_given_yz(1, 1, 0), 0.5, atol=1e-05)
    assert_allclose(p_x_given_yz(0, 0, 1), 0.5333333333333333, atol=1e-05)
    assert_allclose(p_x_given_yz(1, 0, 1), 0.4666666666666667, atol=1e-05)
    assert_allclose(p_x_given_yz(0, 1, 1), 0.5, atol=1e-05)
    assert_allclose(p_x_given_yz(1, 1, 1), 0.5, atol=1e-05)
finally:
    p_x = old_p_x
    p_y = old_p_y
    p_z_given_xy = old_p_z_given_xy
    del old_p_x, old_p_y, old_p_z_given_xy

print("Success!")

<div class="alert alert-success">Run the cell below to compute $P(~X=1~|~Z=1~)$, $P(~X=1~|~Y=1,Z=1~)$, and $P(~X=1~)$. What is the effect of hearing that it rained ($Y=1$) have on your belief that the sprinklers were on ($X=1$), given that you knew the grass was wet? How does this compare to your belief about whether the sprinklers were on in absence of any knowledge about the state of the grass? Why does observing that it rained cause this happen?</div> 

In [ ]:
print("P(X=1 | Z=1)      = {}".format(p_x_given_z(1, 1)))
print("P(X=1 | Y=1, Z=1) = {}".format(p_x_given_yz(1, 1, 1)))
print("P(X=1)            = {}".format(p_x(1)))

YOUR ANSWER HERE

---
## Part F (1 point)

<div class="alert alert-success">The two pieces of evidence we observed (wet grass, and hearing that it rained) both affected the Bayesian network's belief about the sprinklers being on. Compare the pattern of inferences produced by the Bayesian network with those that might result from using a production system of the kind explored in Problem Set 1. You don't need to focus on a specific production system or work out its predictions -- just highlight the aspects of this inference that might be hard to capture using a production system.</div>

YOUR ANSWER HERE

---

Before turning this problem in remember to do the following steps:

1. **Restart the kernel** (Kernel$\rightarrow$Restart)
2. **Run all cells** (Cell$\rightarrow$Run All)
3. **Save** (File$\rightarrow$Save and Checkpoint)

<div class="alert alert-danger">After you have completed these three steps, ensure that the following cell has printed "No errors". If it has <b>not</b> printed "No errors", then your code has a bug in it and has thrown an error! Make sure you fix this error before turning in your problem set.</div>

In [ ]:
print("No errors!")